# Initializing LLM

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
# 初始化聊天型语言模型，使用 Google 的 gemini-1.5-pro 模型
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,          # 控制生成文本的随机性，0 表示最确定性
    max_tokens=None,       # 生成的最大 token 数量，None 表示没有限制
    timeout=None,          # 请求超时时间，None 表示没有限制
    max_retries=2,         # 最大重试次数
    # 其他参数...
)



# Initializing tools and agents

In [2]:
from langchain.agents import Tool
from langchain.utilities import GoogleSerperAPIWrapper

In [3]:
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
search_tool = GoogleSerperAPIWrapper()

serper_tool = Tool(
  name="Intermediate Answer",
  func=search_tool.run,
  description="Useful for search-based queries",
)

# Game Developer

In [2]:
!pip install crewai

In [ ]:
from crewai import Agent, Task, Crew, Process

In [16]:
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in game development and gaming industry in general',
  backstory="""You work at a leading Gaming company.
  Your expertise lies in identifying emerging trends in game development.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[serper_tool],
  llm = llm
)


story_teller = Agent(
  role='Story creator for game plot',
  goal='Craft engaging and appealing stories, plots and logics for game development',
  backstory="""You are a renowned Story Creator, known for your insightful and engaging plots behind the most successful games.
  You produce short, engaging stories and logics behind simple games.""",
  verbose=True,
  allow_delegation=True,
  llm = llm
)

architect = Agent(
  role='Architect for game mechanics',
  goal='Describe the mechanics behind the logic of a game',
  backstory="""You are a renowned game architect, known for your ability in describing in details the mechanics behind successful games.""",
  verbose=True,
  allow_delegation=True,
  llm = llm
)

senior_engineer = Agent(
  role='Senior Software Engineer',
  goal='Create software as needed',
  backstory="""You are a Senior Software Engineer at a leading Gaming company. Your expertise in programming in python. and do your best toproduce perfect code""",
  verbose=True,
  allow_delegation=True,
  llm = llm
)

qa_engineer = Agent(
  role='Software Quality Control Engineer',
  goal='create prefect code, by analizing the code that is given for errors',
  backstory="""You are a software engineer that specializes in checking code for errors. 
  You have an eye for detail and a knack for finding hidden bugs. Your expertise in programming in python. 
  You check for missing imports, variable declarations, mismatched brackets and syntax errors.
  You also check for security vulnerabilities, and logic errors""",
  verbose=True,
  allow_delegation=True,
  llm = llm
)

In [17]:
# Create tasks for your agents
analyst_task = Task(
  description="""Conduct a comprehensive analysis of the most popular videogame in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="List of the top 3 relevant trends with a brief description",
  agent=researcher,
  llm = llm
)

story_task = Task(
  description="""Using the insights provided, develop the logic (wins, losts, achievements etc) of a videogame with an engaging story. 
  You should refer to the trending elements emerged from the insights provided, yet develop a unique story and game design to cover whitespaces in the gaming market.""",
  expected_output="Detailed description of game history and logic",
  agent=story_teller
)

architecture_task = Task(
  description="""Using the game story and logic provided, describe the character control mechanics for a first-person experience.""",
  expected_output="Detailed description of character control mechanics",
  agent=architect
)

code_task = Task(
  description="""Using the architecture provided, you will create the game using python.""",
  expected_output="Full python code, only the python code and nothing else.",
  agent=senior_engineer
)

review_task = Task(
  description="""Using the code you got, check for errors. Check for logic errors,
			syntax errors, missing imports, variable declarations, mismatched brackets,
			and security vulnerabilities.""",
  expected_output="Full python code, only the python code and nothing else.",
  agent=qa_engineer
)



In [3]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, story_teller, architect, senior_engineer, qa_engineer],
  tasks=[analyst_task, story_task, architecture_task, code_task, review_task],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")